In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn import preprocessing
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
dataset = "../../all"
data = pd.read_csv(dataset+'/training_set.csv')
list(data)

['object_id', 'mjd', 'passband', 'flux', 'flux_err', 'detected']

In [3]:
#first make the mjd integer and if we have duplicated values take the means
#put all this is a black box train it on GP then out put to a file
#input to model object and pace
#notes : TimeSeriesSplit

In [11]:
objects = data.drop_duplicates(subset=['object_id'])['object_id']
data['mjd'] = data['mjd'].astype(int)

In [20]:
def generateY(mjd,interval,maxVal):
    y = list(range(0,maxVal+1,interval))
    mjd = mjd.drop_duplicates()
    l = mjd.values.tolist()
    mjdToPredict = list(set(y) - set(l))
    return mjdToPredict

In [21]:
def gaussianFill(objectDf,maxVal,interval):
    grouped = data.groupby(['mjd','passband']).mean()
    grouped.reset_index(level=['passband','mjd'],inplace=True)
    grouped.mjd = grouped.mjd - grouped.mjd.min()
    predict = generateY(grouped.mjd,interval,maxVal)
    result = pd.DataFrame({'object_id':[],'mjd':[],'passband':[],'flux':[],'flux_err':[]})
    frames = []
    for i in range(0,6):
        flux = []
        filtered = grouped[grouped['passband'] == i ]
        X_train = filtered[['mjd']]
        y_train = filtered[['flux','flux_err']]
        kernel = DotProduct() + WhiteKernel()
        gaussian = GaussianProcessRegressor()
        gaussian.fit(X_train,y_train)
        flux = gaussian.predict(pd.DataFrame(predict))
        ids = [objectDf['object_id'].iloc[0]]*len(flux)
        passband = [i]*len(flux)
        df = pd.DataFrame({'object_id':ids,'passband':passband,'mjd':predict,'flux':[i[0] for i in flux],'flux_err':[i[1] for i in flux]})
        frames = frames + [df]
    result = pd.concat(frames)
    return pd.concat(frames)

In [22]:
result = pd.DataFrame({'object_id':[],'mjd':[],'passband':[],'flux':[],'flux_err':[]})
predictions = []
for o in objects:
    predictions.append(gaussianFill(data[data['object_id'] == o ],data.mjd.max() - data.mjd.min(),1))
    break;
result = pd.concat(predictions)
result.to_csv(dataset+'/flux_predictions.csv')